In [1]:
!pip install datasets

In [2]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer

In [3]:
from datasets import Dataset, DatasetDict
from tensorflow.keras.callbacks import EarlyStopping

In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score

In [5]:
!gdown 1OS2Aurb2oKmooCeYwYw0cnp542t7dyHo
!gdown 1Oibb4sFzJNLt0iBWOD4LAIKaWT7cCNT1

Downloading...
From: https://drive.google.com/uc?id=1OS2Aurb2oKmooCeYwYw0cnp542t7dyHo
To: /content/test_data.csv
100% 565k/565k [00:00<00:00, 145MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Oibb4sFzJNLt0iBWOD4LAIKaWT7cCNT1
To: /content/train_data.csv
100% 2.24M/2.24M [00:00<00:00, 108MB/s]


In [6]:
train_df = pd.read_csv("train_data.csv")
train_df.head()

,confidence,social_karma,syntax_ari,lex_liwc_WC,lex_liwc_Analytic,lex_liwc_Clout,lex_liwc_Authentic,lex_liwc_Tone,lex_liwc_WPS,lex_liwc_Sixltr,...,lex_dal_min_imagery,lex_dal_avg_activation,lex_dal_avg_imagery,lex_dal_avg_pleasantness,social_upvote_ratio,social_num_comments,syntax_fk_grade,sentiment,clean_text,target
0,1.000000,3,5.311000,89,63.33,10.84,98.92,3.83,17.80,12.36,...,1.0,1.74515,1.42143,1.83704,0.58,27,6.376667,0.052071,reluctant start one really ashamed ask help on...,1
1,1.000000,3,4.220972,108,19.76,11.45,95.96,1.00,15.43,12.96,...,1.0,1.71501,1.57333,1.85046,1.00,0,5.593690,-0.022917,severe addict overdose young addictionalcoholi...,1
2,0.500000,15,5.548481,79,12.35,44.98,69.44,1.00,15.80,16.46,...,1.0,1.80983,1.48267,1.86705,0.91,5,6.703646,-0.460000,feel sad feel sad pray get help desperately ne...,1
3,0.833333,18,4.352143,81,93.97,40.24,99.00,25.77,16.20,12.35,...,1.0,1.70892,1.84444,1.81327,0.85,11,5.150095,0.048636,sleep car month month back foot stay parking s...,1
4,0.833333,4,7.366712,135,40.97,72.32,75.74,1.00,27.00,12.59,...,1.0,1.75929,1.58319,1.84228,1.00,5,7.778319,0.031250,speed try catch continue yell face girl turn a...,0


In [7]:
test_df = pd.read_csv("test_data.csv")
test_df.head()

,confidence,social_karma,syntax_ari,lex_liwc_WC,lex_liwc_Analytic,lex_liwc_Clout,lex_liwc_Authentic,lex_liwc_Tone,lex_liwc_WPS,lex_liwc_Sixltr,...,lex_dal_min_imagery,lex_dal_avg_activation,lex_dal_avg_imagery,lex_dal_avg_pleasantness,social_upvote_ratio,social_num_comments,syntax_fk_grade,sentiment,clean_text,target
0,0.600000,10,0.852727,31,2.36,16.66,99.00,83.22,6.20,12.90,...,1.0,1.69888,1.45806,2.03517,0.91,3,2.002182,0.391667,abuse end year ago life good get marry someone...,0
1,1.000000,3,3.754597,118,43.24,11.78,97.74,6.73,39.33,12.71,...,1.0,1.74280,1.55769,1.92918,1.00,5,4.443710,0.057273,hi health issue miss lot school eventually sta...,1
2,0.666667,195,0.421475,57,1.00,36.27,2.09,1.00,11.40,3.51,...,1.0,1.72157,1.56786,1.87074,0.89,31,2.515541,-0.250000,know want curse hit angry come sister home wan...,1
3,0.800000,4,1.513118,91,23.17,22.08,81.48,1.10,18.20,8.79,...,1.0,1.69089,1.48250,1.87478,0.75,4,2.523871,-0.093519,never feel uncomfortable scared night put hand...,1
4,1.000000,1,6.293125,109,26.34,79.55,56.75,13.38,21.80,21.10,...,1.0,1.76262,1.53895,1.96475,0.60,3,6.348214,0.150000,deal ptsd rape recently find someone think fri...,1


In [8]:
train_df = train_df[train_df['confidence'] >= 0.8].reset_index(drop=True)

In [9]:
train_df.shape

(1832, 111)

In [10]:
test_df = test_df[test_df['confidence'] >= 0.8].reset_index(drop=True)
test_df.shape

(465, 111)

In [11]:
train_df = train_df[['clean_text', 'target']]
train_df.head()

,clean_text,target
0,reluctant start one really ashamed ask help on...,1
1,severe addict overdose young addictionalcoholi...,1
2,sleep car month month back foot stay parking s...,1
3,speed try catch continue yell face girl turn a...,0
4,want keep contact still thing together friend ...,0


In [12]:
test_df = test_df[['clean_text', 'target']]

In [13]:
train_df = train_df.rename(columns={"clean_text": "text", "target":"label"})
train_df.head()

,text,label
0,reluctant start one really ashamed ask help on...,1
1,severe addict overdose young addictionalcoholi...,1
2,sleep car month month back foot stay parking s...,1
3,speed try catch continue yell face girl turn a...,0
4,want keep contact still thing together friend ...,0


In [14]:
test_df = test_df.rename(columns={"clean_text": "text", "target":"label"})
test_df.head()

,text,label
0,hi health issue miss lot school eventually sta...,1
1,never feel uncomfortable scared night put hand...,1
2,deal ptsd rape recently find someone think fri...,1
3,check rkratom info personally swear edit seem ...,0
4,know two week girlfriend single mom two kid ex...,1


In [15]:
train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)

In [16]:
train_ds[10]

{'text': 'wish luck homie edit get job go pay ask think hire manager figure homeless ask dependable transportation place residence never interview ask place live question stumble might give inclination homeless',
 'label': 0}

In [17]:
def order(inp):
  '''
  This function will group all the inputs of BERT
  into a single dictionary and then output it with
  labels.
  '''
  data = list(inp.values())
  return {
      'input_ids': data[1],
      'attention_mask': data[2],
      'token_type_ids': data[3]
  }, data[0]

def config_datasets(train_ds, test_ds, tokenizer):
  def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

  train_ds = train_ds.map(tokenize, batched=True, batch_size=None)
  test_ds = test_ds.map(tokenize, batched=True, batch_size=None)

  train_ds.set_format('tf', columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])
  test_ds.set_format('tf', columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])

  BATCH_SIZE = 64



  # converting train split of `emotions_encoded` to tensorflow format
  train_dataset = tf.data.Dataset.from_tensor_slices(train_ds[:])
  # set batch_size and shuffle
  train_dataset = train_dataset.batch(BATCH_SIZE)
  # map the `order` function
  train_dataset = train_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

  # ... doing the same for test set ...
  test_dataset = tf.data.Dataset.from_tensor_slices(test_ds[:])
  test_dataset = test_dataset.batch(BATCH_SIZE)
  test_dataset = test_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)
  return train_dataset, test_dataset

In [18]:
def get_report(y_test, y_pred):
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1score = f1_score(y_test, y_pred)
  return {'Accuracy' : accuracy, 'Precision': precision, 'Recall' : recall, 'F1-Score' : f1score}

In [19]:
class BERTForClassification(tf.keras.Model):

  def __init__(self, bert_model, num_classes):
    super().__init__()
    self.bert = bert_model
    # self.dropout = tf.keras.layers.Dropout(0.2)  # Dropout layer with 0.2 rate
    self.fc = tf.keras.layers.Dense(1, activation='sigmoid')

  def call(self, inputs):
    x = self.bert(inputs)[1]  # Access pooled output from BERT
    # x = self.dropout(x)  # Apply dropout to the pooled output
    # x = self.h1(x)
    return self.fc(x)

In [20]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = TFAutoModel.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [21]:
classifier = BERTForClassification(model, num_classes=2)
train_dataset, test_dataset = config_datasets(train_ds, test_ds, tokenizer)
classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

Map:   0%|          | 0/1832 [00:00<?, ? examples/s]

Map:   0%|          | 0/465 [00:00<?, ? examples/s]

In [22]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = classifier.fit(
    train_dataset,
    epochs=10,
    validation_data=(test_dataset),
    callbacks=[early_stop]
)

Epoch 1/10
29/29 [==============================] - 138s 3s/step - loss: 0.6600 - accuracy: 0.6141 - val_loss: 0.6098 - val_accuracy: 0.7032
Epoch 2/10
29/29 [==============================] - 67s 2s/step - loss: 0.5007 - accuracy: 0.7729 - val_loss: 0.4964 - val_accuracy: 0.7656
Epoch 3/10
29/29 [==============================] - 68s 2s/step - loss: 0.3527 - accuracy: 0.8603 - val_loss: 0.4871 - val_accuracy: 0.7548
Epoch 4/10
29/29 [==============================] - 67s 2s/step - loss: 0.2447 - accuracy: 0.9110 - val_loss: 0.4684 - val_accuracy: 0.7677
Epoch 5/10
29/29 [==============================] - 68s 2s/step - loss: 0.1551 - accuracy: 0.9541 - val_loss: 0.6510 - val_accuracy: 0.7548
Epoch 6/10
29/29 [==============================] - 67s 2s/step - loss: 0.1906 - accuracy: 0.9405 - val_loss: 0.8689 - val_accuracy: 0.7183
Epoch 7/10
29/29 [==============================] - 67s 2s/step - loss: 0.2126 - accuracy: 0.9138 - val_loss: 0.5743 - val_accuracy: 0.7763
Epoch 8/10
29/29 [=

In [23]:
y_pred = classifier.predict(test_dataset)
y_pred = (y_pred > 0.5).astype(int)


8/8 [==============================] - 61s 557ms/step


In [24]:
get_report(test_ds['label'], y_pred)

{'Accuracy': 0.7677419354838709,
 'Precision': 0.7538461538461538,
 'Recall': 0.8166666666666667,
 'F1-Score': 0.784}